In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from alphadia.extraction import data

In [2]:

raw_name = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_AID8_10_S4-G9_1_667.d'
dia_data = data.TimsTOFTranspose(raw_name)


100%|██████████| 17438/17438 [00:12<00:00, 1350.86it/s]


In [3]:
jit_data = dia_data.jitclass()

In [4]:
# test get_frame_indices

def fuzz_get_frame_indices(jit_data):

    start_index = np.random.randint(0, jit_data.rt_values.shape[0], size=1)[0]
    stop_index = np.random.randint(start_index, jit_data.rt_values.shape[0], size=1)[0]

    rt = jit_data.rt_values[[start_index, stop_index]]
    frame_indices = jit_data.get_frame_indices(rt, 16)

    # make sure the shape matches
    assert frame_indices.shape == (2,)

    assert np.all(frame_indices >= 0)
    assert np.all(frame_indices < jit_data.rt_values.shape[0])
    assert np.all(frame_indices[1] >= frame_indices[0])

    cycle_offset = (frame_indices-jit_data.zeroth_frame) % jit_data.cycle.shape[1]

    # make sure that the start and stop are at the start of a cycle
    assert np.all(cycle_offset == 0)

    cycle_indices = (frame_indices-jit_data.zeroth_frame) // jit_data.cycle.shape[1]
    cycle_len = cycle_indices[1] - cycle_indices[0]

    # make sure the cycle length is a multiple of 16
    assert cycle_len % 16 == 0

for i in range(1000):
    fuzz_get_frame_indices(jit_data)

In [5]:
# test get_frame_indices_tolerance

def fuzz_get_frame_indices_tolerance(jit_data):

    rt = jit_data.rt_values[np.random.randint(0, jit_data.rt_values.shape[0], size=1)][0]
    rt_tolerance = np.random.uniform(0, 300, size=1)[0]

    frame_indices = jit_data.get_frame_indices_tolerance(rt, rt_tolerance, 16)

    # make sure the shape matches
    assert frame_indices.shape == (1, 3)

    frame_indices = frame_indices[0,[0,1]]

    assert np.all(frame_indices >= 0)
    assert np.all(frame_indices < jit_data.rt_values.shape[0])
    assert np.all(frame_indices[1] >= frame_indices[0])

    cycle_offset = (frame_indices-jit_data.zeroth_frame) % jit_data.cycle.shape[1]

    # make sure that the start and stop are at the start of a cycle
    assert np.all(cycle_offset == 0)

    cycle_indices = (frame_indices-jit_data.zeroth_frame) // jit_data.cycle.shape[1]
    cycle_len = cycle_indices[1] - cycle_indices[0]

    # make sure the cycle length is a multiple of 16
    assert cycle_len % 16 == 0

for i in range(1000):
    fuzz_get_frame_indices_tolerance(jit_data)


In [6]:
# test get_scan_indices

def fuzz_get_scan_indices(jit_data):

    start_index = np.random.randint(0, jit_data.mobility_values.shape[0], size=1)[0]
    stop_index = np.random.randint(start_index, jit_data.mobility_values.shape[0], size=1)[0]

    mobility = jit_data.mobility_values[[start_index, stop_index]]
    scan_indices = jit_data.get_scan_indices(mobility, 16)

    # make sure the shape matches
    assert scan_indices.shape == (2,)

    assert np.all(scan_indices >= 0)
    assert np.all(scan_indices <= jit_data.mobility_values.shape[0])
    assert np.all(scan_indices[1] >= scan_indices[0])

    scan_len = scan_indices[1] - scan_indices[0]

    # make sure the cycle length is a multiple of 16
    assert scan_len % 16 == 0

for i in range(10):
    fuzz_get_scan_indices(jit_data)

In [7]:
# test get_scan_indices_tolerance
# test get_frame_indices_tolerance

def fuzz_get_scan_indices_tolerance(jit_data):

    mobility = jit_data.mobility_values[np.random.randint(0, jit_data.mobility_values.shape[0], size=1)][0]
    mobility_tolerance = np.random.uniform(0, 0.1, size=1)[0]

    scan_indices = jit_data.get_scan_indices_tolerance(mobility, mobility_tolerance, 16)

    # make sure the shape matches
    assert scan_indices.shape == (1, 3)

    scan_indices = scan_indices[0,[0,1]]

    assert np.all(scan_indices >= 0)

    if not np.all(scan_indices <= jit_data.mobility_values.shape[0]):
        print(mobility, mobility_tolerance)
        print(scan_indices)
    assert np.all(scan_indices[1] >= scan_indices[0])

    scan_len = scan_indices[1] - scan_indices[0]

    # make sure the cycle length is a multiple of 16
    assert scan_len % 16 == 0

for i in range(1000):
    fuzz_get_scan_indices_tolerance(jit_data)

In [9]:
from tqdm import tqdm
import numba as nb
import numpy as np
from alphadia.extraction import utils
import matplotlib.pyplot as plt

def fuzz_jit_data(jit_data):
    min_mz = jit_data.mz_values.min()
    max_mz = jit_data.mz_values.max()
    use_q = np.random.randint(0, 2, dtype=bool)
    
    if use_q:
        _precursor = np.random.uniform(jit_data.quad_min_mz_value, jit_data.quad_max_mz_value, size=1)[0]
        precursor = np.array([[_precursor, _precursor]], dtype=np.float32)

    else:
        precursor = np.array([[-1.,-1.]], dtype=np.float32)

    mz = np.sort(np.random.uniform(min_mz, max_mz, size=10).astype(np.float32))
    mz_tolerance = 150#np.random.uniform(0, 200, size=1)[0]

    rt = jit_data.rt_values[np.random.randint(0, jit_data.rt_values.shape[0], size=1)][0]
    rt_tolerance = np.random.uniform(0, 300, size=1)[0]

    mobility = jit_data.mobility_values[np.random.randint(0, jit_data.mobility_values.shape[0], size=1)][0]
    mobility_tolerance = np.random.uniform(0, 0.1, size=1)[0]

    scan_limits = jit_data.get_scan_indices_tolerance(
        mobility,
        mobility_tolerance
    )

    frame_limits = jit_data.get_frame_indices_tolerance(
        rt,
        rt_tolerance
    )

    dense, precursor_index = jit_data.get_dense(
        frame_limits,
        scan_limits,
        mz,
        mz_tolerance,
        precursor,
        True,
    )

    print(
        frame_limits,
        frame_limits.dtype,
        scan_limits,
        scan_limits.dtype,
        mz,
        mz.dtype,
        mz_tolerance,
        precursor,
        precursor.dtype,
        True,
    )

    

    if dense.shape[0] == 0:
        return

for i in tqdm(range(1)):
    fuzz_jit_data(jit_data)

100%|██████████| 1/1 [00:00<00:00, 78.05it/s]

[[2485 3637    1]] int64 [[459 523   1]] int64 [ 185.12119  374.02203  510.31326  601.63464  685.67334  856.825
  917.9102  1307.4122  1554.3955  1602.5912 ] float32 150 [[762.5863 762.5863]] float32 True
